<a href="https://colab.research.google.com/github/dhwan-dj/S4pjt_NLP_koBERT/blob/main/S4_PJT_Sentiment_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 개요 : KoBERT를 이용하여 한국어 문장을 여러 클래스로 분류하는 모델을 만드는 것인데, 공포, 놀람, 분노, 슬픔, 중립, 행복, 혐오와 같은 감정이 느껴지는 짧은 대화 텍스트를 각각 어떠한 감정의 텍스트인지 분류하는 모델을 만드는 것이다. 예를 들어 "앗 깜작이야!" 라는 문장을 입력하면 '놀람'이라는 클래스로 예측을 하도록 학습시키는 것
- 코드는 KoBERT 깃허브에 있는 네이버 영화평 이중분류 예시 코드를 바탕으로 작성
- 참고 블로그 (https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab)

# 1. Colab 환경설정

- 라이브러리, 모듈 설치, koBERT 모델 불러오기
- 예시 코드의 Dataset = '한국어 감정 정보가 포함된 단발성 대화'
- 본 코드의 Dataset : 감성대화말뭉치(https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86)

<세팅 및 파라미터>  

    Python >= 3.6
    PyTorch >= 1.70
    Transformers = 3.0.2
    Colab
    batch size = 64
    epochs = 10

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689000 sha256=2cd3dcf597dcac0feb44575ce27e116e23be78da27faf500b9e026216a5a6518
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 깃허브에서 KoBERT 파일들 다운로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-5k6t13qj
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-5k6t13qj
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 877.9/881.9 MB 147.9 MB/s eta 0:00:01tcmalloc: large alloc 1102397440 bytes == 0x3b1d8000 @  0x7fa378624680 0x7fa378644da2 0x5f714c 0x64d800 0x527022 0x5048

In [ ]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf 의 kobert_tokenizer 폴더를 다운받는 코드.
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [3]:
import torch             # pytorch framework
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
# https://github.com/SKTBrain/KoBERT/tree/master/kobert/utils 의 utils.py 파일에서 get_tokenizer 메서드를 불러오는 코드
from kobert.pytorch_kobert import get_pytorch_kobert_model
# https://github.com/SKTBrain/KoBERT/tree/master/kobert 의 pytorch_kobert.py 파일에서 get_pytorch_kobert_model 메서드를 불러오는 코드

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
#GPU 사용
device = torch.device("cuda:0")

In [33]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


- BERT는 이미 누군가가 학습해둔 모델을 사용한다(pre-trained model)는 것을 뜻한다. 따라서 사용하는 model과 tokenizer는 항상 mapping 관계여야 한다. 예를 들어서 U 팀이 개발한 BERT를 사용하는데, V팀이 개발한 BERT의 tokenizer를 사용하면 model은 텍스트를 이해할 수 없다. U팀의 BERT의 토크나이저는 '우리'라는 단어를 23번으로 int encoding하는 반면에, V라는 BERT의 tokenizer는 '우리'라는 단어를 103번으로 int encoding해 단어와 mapping 되는 정보 자체가 달라지기 때문이다. 
(https://hoit1302.tistory.com/159)

# 2. 데이터셋 로드, 전처리

In [6]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd
train = pd.read_excel('/content/drive/MyDrive/data/sentiment_train.xlsx')
val = pd.read_excel('/content/drive/MyDrive/data/sentiment_val.xlsx')

In [35]:
df = pd.concat([train,val])

In [36]:
len(df) # 문장 = 58,271개

58271

In [37]:
df.sample(5)

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
10020,10021,청년,여성,대인관계,해당없음,당황,고립된,내가 평소에 같이 스터디하는 모임에서 안 좋은 일이 있었어.,그랬군요. 어떤 안 좋은 일인지 구체적으로 말씀해 주실 수 있나요?,갑자기 나를 둘러싸더니 그렇게 행동하지 말라는 거야. 거의 옛날 학창 시절에 왕따를...,갑자기 그런 일을 당하시다니 당황스러우셨겠어요.,응 맞아. 다음 주에도 그 사람들과 만나야 하는데 어떻게 해야 할지 감이 안 와.,그런 안 좋은 일을 만든 사람들과 다시 봐야 한다니 정말 고민이 많이 되겠어요.
26793,26794,청년,여성,대인관계,해당없음,당황,혼란스러운,내가 자취를 하고 있는데 내 친구가 같이 살면 안되겠냐고 해서 너무 혼란스러워,친구분이 같이 살자고 해서 많이 혼란스러우시겠어요.,같이 사는 건 무리일 것 같은데 어떻게 얘기해야 할지 고민 중이야.,거절하는 방법을 고민하고 계시군요. 이 문제를 해결하기 위해 어떤 방법이 가장 좋을까요?,사실 그대로 얘기하는 게 좋을것 같아.,솔직하게 이야기해 보려고 하시는군요.
24221,24222,노년,남성,재정,만성질환 유,불안,걱정스러운,수술비를 마련해야 하는데 대출이 안 될까 봐 걱정이야.,수술비 때문에 걱정이 많으시군요.,꼭 수술을 해야 한다는데 정말 막막해.,어떻게 하면 지금의 상황을 극복 할 수 있을까요?,가족들에게 말을 하고 한번 도움을 청해보려 해.,아들과 대화가 잘 되어 상황이 좋게 해결됐으면 좋겠어요.
36713,36714,청소년,여성,학교폭력/따돌림,해당없음,분노,툴툴대는,선생님은 왜 내 말을 안 신뢰해주시는걸까?,선생님이 말을 신뢰해 주지 않으신다니 무슨 일이신가요?,내가 친구들에게 따돌림 당하는 것 같다고 하니까 안 들어주셔.,어찌 말하면 진정성있게 들릴 수 있을까요?,전과 다르게 진심을 표현하여 자세하게 말씀드려봐야겠어.,선생님이 신뢰하고 이야기를 잘 들어주길 바랄게요.
18157,18158,청소년,남성,가족관계,해당없음,슬픔,비통한,이렇게 추운 날 아빠가 그렇게 먼 거리를 걸어서 오셨대. 너무 속상해.,추운 날에 아빠가 먼 거리를 걸어 오셔서 정말 속상하시겠어요.,내가 힘이 되어주고 싶어.,생각 중인 방법이 있으신가요?,아르바이트를 해서 내 용돈을 스스로 벌 생각이야.,용돈을 스스로 버실 생각이시군요.


- 6번째 칼럼 '감정_대분류'를 라벨 삼아 입력문장의 감정을 예측하기로 한다.

In [22]:
df.감정_대분류.value_counts()

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: 감정_대분류, dtype: int64

In [38]:
# 주관적으로 긍정에서 부정으로 숫자 라벨을 매겨보았다.
df.loc[(df['감정_대분류'] == "기쁨"), 'sentiment'] = '0'    #기쁨 => 0
df.loc[(df['감정_대분류'] == "당황"), 'sentiment'] = '1'    #당황 => 1
df.loc[(df['감정_대분류'] == "불안"), 'sentiment'] = '2'    #불안 => 2
df.loc[(df['감정_대분류'] == "분노"), 'sentiment'] = '3'    #분노 => 3
df.loc[(df['감정_대분류'] == "슬픔"), 'sentiment'] = '4'    #슬픔 => 4
df.loc[(df['감정_대분류'] == "상처"), 'sentiment'] = '5'    #상처 => 5

In [39]:
# 문장과 문장의 감정을 리스트로 저장
data_list = []
for q, label in zip(df['사람문장1'], df['sentiment']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [40]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '3']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어.', '3']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아.', '2']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어.', '5']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어.', '5']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐.', '4']
['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.', '1']


# 3. Train과 Test Set 준비

In [41]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.15, random_state=0)

In [42]:
print(len(dataset_train))
print(len(dataset_test))

49530
8741


# 4. koBERT의 입력 데이터 형태로 만들기
- 모델의 입력으로 활용하기 위해 토큰화, 라벨과 문장의 인덱스 인코딩, 패딩 등이 필요하다.
- 예시 코드의 입력데이터 형태로 바꿔주는 클래스 활용

In [30]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
      transform = nlp.data.BERTSentenceTransform(
          bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)   # BERTSentenceTransform 으로 토큰화, 패딩
      
      self.sentences = [transform([i[sent_idx]]) for i in dataset]
      self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
      return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
      return (len(self.labels))

In [31]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [43]:
# BERTTokenizer와 위의 Class로 Tokenize, Padding
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


- 토큰화와 패딩이 잘 이루어졌을까?
- 데이터는 3개의 array인데, 첫번째는 패딩된 시퀀스, 두번째는 길이와 타입, 세번째는 어텐션 마스크 시퀀스라고 한다.
- BERT에 데이터가 입력되었을 때 어텐션 함수가 적용되어 연산이 된다. 이때 1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다고 알려주는 데이터가 있어야 하는데 그게 바로 어텐션 마스크 시퀀스 라고 한다. (아직 이해x)

In [44]:
data_train[0]

(array([   2, 1261, 7318, 6941, 7096, 3869,  784, 5579,  889, 6122, 6198,
        4688, 7126, 7120, 4304, 4213, 6896, 1934, 4627, 6897,  784, 5573,
         517, 6976, 6797, 6323, 6072, 6855,  517,   54,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(31, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [45]:
# torch 형식의 dataset을 만들어준다.
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5. koBERT 학습모델 만들기
- num_classes = 6 (6개의 감정 클래스)

In [46]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [47]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. BERT model 학습시키기

In [48]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-48-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/774 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9431135654449463 train acc 0.109375
epoch 1 batch id 201 loss 1.4557873010635376 train acc 0.25404228855721395
epoch 1 batch id 401 loss 1.2469488382339478 train acc 0.3852478179551122
epoch 1 batch id 601 loss 1.1654717922210693 train acc 0.4378639767054909
epoch 1 train acc 0.4647027866880514


<ipython-input-48-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/137 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5935157328861709


  0%|          | 0/774 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.0171887874603271 train acc 0.6875
epoch 5 batch id 201 loss 0.5310825109481812 train acc 0.7300217661691543
epoch 5 batch id 401 loss 0.7008195519447327 train acc 0.740843204488778
epoch 5 batch id 601 loss 0.6874670386314392 train acc 0.7470361896838602
epoch 5 train acc 0.7462423705782767


  0%|          | 0/137 [00:00<?, ?it/s]

epoch 5 test acc 0.5938054843164332


- Epochs 5 학습은 약 47분이 걸렸고, train dataset에 대해서는 0.747, test dataset에 대해서는 0.594의 정확도를 기록했다.

- 이거 좀 많이 낮은데;;

- 참고블로그에서는 정확도가 높았는데, 이에 대한 설명 :
train dataset에 대해서는 0.979, test dataset에 대해서는 0.918의 정확도를 기록
이렇게 높은 정확도를 기록하는 이유는 바로 데이터셋에 있다. 진행하고 있는 프로젝트에서 해당 프로젝트의 성격에 맞게 3만 7천 여개 정도의 데이터를 직접 재분류하고 있는데, 사람이 생각하기에는 분명히 같은 내용으로 인지되는 문장이지만 사소한 단어를 한 두개 없애서, 어떨 때는 중요한 단어를 제거해서, 짧게 잘라서, 길게 늘여뜨려서, 문장 부호를 다르게, 감탄사를 추가해서 등등 중복인 듯 중복 아닌 데이터로 학습시켰기 때문에 이렇게 높은 정확도가 나온 것이라 판단된다.
(https://hoit1302.tistory.com/159)

# 7. 새로운 문장 테스트
- 새로운 문장을 koBERT의 입력 형식으로 바꿔주는 predict 함수를 정의한다.

In [49]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("불안이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [51]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 피곤하다
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 0


# **Result**
- 학습데이터 정확도
- 테스트데이터 정확도
- F1, Precision, Recall 점수를 계산
- 새로운 문장에 대한 정확도

- 한계점

- 개선점

- 기대효과